# Plots for my PhD Thesis

All those plots come from my different researches. The intention is to have them coherently designed.

### Required packages and Constants

In [21]:
using Plotly

In [22]:
L_FSIZE = 18
T_FSIZE = 14
CMAP = "viridis"
CMAP_R = "viridis_r"
HG = 3.5 #inches
WD = 5 #inches
LWD = 1.5; #points
BAR_BLUE = (0.4,0.6,0.8)
BG_GREY = "0.95"

"0.95"

## Plots for the background in Quantum Metrology

## Plot for accuracy bound in the vicinity of Dicke states

In [1]:
function plotVD_precisionOnTheta()

    # Open data group of the plot
    group = g_open(h5open("plotsData.h5"),"VicinityDicke/PrecisionOverTheta")

    theta = d_read(group,"theta")
    precision = d_read(group, "precision")

    theta_max = a_read(group,"theta_max")
    precision_max = a_read(group,"precision_max")

    plot([minimum(theta),maximum(theta)],
        [1,1])

end

  likely near /home/iapellaniz/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/iapellaniz/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/iapellaniz/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/iapellaniz/.julia/v0.5/IJulia/src/kernel.jl:31


plotVD_precisionOnTheta (generic function with 1 method)

  likely near /home/iapellaniz/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/iapellaniz/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/iapellaniz/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/iapellaniz/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/iapellaniz/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/iapellaniz/.julia/v0.5/IJulia/src/execute_request.jl


In [2]:
plotVD_precisionOnTheta

plotVD_precisionOnTheta (generic function with 1 method)

### Plot evolution

### Plot versus QFI of known states

### Plot contour

## Plots for the Optimal Bounds for QFI

Get image for the spin-squeezing bound from blender(c), still it is not possible to do this on matplotlib. This is because the smoothness needed at the border. Image file `data/LT_spsq2d_4.png`.

### Op. bound for spin-squeezing inequality 

## PLOTS 